In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data as dt
import torchvision.transforms as transforms
import time
import os
import random
from PIL import Image
from youtube_scraper import download_csv_audio
from preprocessor import autoencoder_hilbert_data, autoencoder_spectrogram_data

In [2]:
og = os.getcwd()

In [4]:
def get_autoencoder_data(csv_path, win_len = 2048, hilbert = True, set_percent = 0.1):
    download_csv_audio(csv_path)                                                                    #Download all the youtube links as .wav files
    os.chdir(og+"/Downloads/Audios/")
    n = 0
    counter = 0
    for file in os.listdir(og+'/Downloads/Audios'):
        n += 1                                                                                          #where the youtube files were saved
        print(f"Iteration{n}")
        print(file)
        if file.endswith(".wav"):                                                                   #For each .wav file in the downloaded path
            print("executed1")
            if hilbert: 
                newpath = og + '/Data/Hilbert/Autoencoder/'                                                 #If hilbert is required, save the .wav files as hilbert curve data
                print("Executed2")
                data = autoencoder_hilbert_data(file, win_len = win_len)
            else:                                                                                                                #Else, save the .wav files as spectrogram data
                newpath = og + 'Data/Spectrogram/Autoencoder/'
                print("Executed 3")                                                                                       
                data = autoencoder_spectrogram_data(file, win_len = win_len)
            
            print('executed4')
        
            #print(data)
            try:
                for array in data:
                    counter += 1
                    norm = (array - np.min(array))/(np.max(array) - np.min(array))*255
                    norm = norm.astype(np.uint8)
                    im = Image.fromarray(norm)
                    im.save(newpath + str(counter) + ".png")
            except:
                continue
    
    autoencoder_dataset(newpath, set_percent = 0.1)
    
    

In [14]:
def autoencoder_dataset(path, set_percent = 0.1):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5/3))])   #Make sure images are tensors, lie within 0 and 1
    dataset = torchvision.datasets.DatasetFolder.make_dataset(path, class_to_idx = {'stuff':1})  #Turn the images into a dataset
    set_size = int(set_percent*len(dataset))                                                        #Length of test and validation sets
    return dt.random_split(dataset, [len(dataset) - 2*set_size, set_size, set_size])                #Return datasets in order of training set, validation set, test setde

In [15]:
train, val, test = autoencoder_dataset('Data\Hilbert\Autoencoder', set_percent = 0.1)

ValueError: Both extensions and is_valid_file cannot be None or not None at the same time

In [ ]:
download_csv_audio('piece.csv')